In [16]:
# Data analysis
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as sk
import keras

# Bioinformatics
from BCBio.GFF import GFFExaminer

# Convenience
from tqdm import tqdm_notebook
from math import ceil, floor

## Playing with GTF files

In [4]:
gtf_filename = "../Labels/Homo_sapiens.GRCh38.91.gtf"

In [5]:
ann_df = pd.read_csv(gtf_filename, sep="\t", comment="#", header=None)

/home/domain/explover/ChromatinClustering/env/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [15]:
ann_df[2].unique()

array(['gene', 'transcript', 'exon', 'CDS', 'start_codon', 'stop_codon',
       'five_prime_utr', 'three_prime_utr', 'Selenocysteine'], dtype=object)

In [24]:
ann_df.iloc[0][8]

'gene_id "ENSG00000223972"; gene_version "5"; gene_name "DDX11L1"; gene_source "havana"; gene_biotype "transcribed_unprocessed_pseudogene";'

In [22]:
len(ann_df)

2612843

In [44]:
ann_df.head(10)

,0,1,2,3,4,5,6,7,8
0,1,havana,gene,11869,14409,.,+,.,"gene_id ""ENSG00000223972""; gene_version ""5""; g..."
1,1,havana,transcript,11869,14409,.,+,.,"gene_id ""ENSG00000223972""; gene_version ""5""; t..."
2,1,havana,exon,11869,12227,.,+,.,"gene_id ""ENSG00000223972""; gene_version ""5""; t..."
3,1,havana,exon,12613,12721,.,+,.,"gene_id ""ENSG00000223972""; gene_version ""5""; t..."
4,1,havana,exon,13221,14409,.,+,.,"gene_id ""ENSG00000223972""; gene_version ""5""; t..."
5,1,havana,transcript,12010,13670,.,+,.,"gene_id ""ENSG00000223972""; gene_version ""5""; t..."
6,1,havana,exon,12010,12057,.,+,.,"gene_id ""ENSG00000223972""; gene_version ""5""; t..."
7,1,havana,exon,12179,12227,.,+,.,"gene_id ""ENSG00000223972""; gene_version ""5""; t..."
8,1,havana,exon,12613,12697,.,+,.,"gene_id ""ENSG00000223972""; gene_version ""5""; t..."
9,1,havana,exon,12975,13052,.,+,.,"gene_id ""ENSG00000223972""; gene_version ""5""; t..."


In [52]:
max(ann_df[ann_df[1] == "ensembl"][4])

248912795

In [46]:
len(ann_df[ann_df[1] == "ensembl"])

248741

In [42]:
for ind, row in tqdm_notebook(ann_df.iloc[:10000].iterrows()):
    if row[2] == "gene":
        start = ceil(row[3] / 100)
        stop = ceil(row[4] / 100)
        print(start, stop)

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

119 145
145 296
174 175
296 312
304 306
346 361
525 534
576 642
655 716
893 1338
896 912
1311 1349
1352 1359
1377 1380
1398 1404
1415 1739
1578 1579
1605 1616
1827 1842
1853 1955
1879 1880
2579 3597
3480 3484
3589 3661
3654 5230
4399 4403
4508 4517
4872 4900
4913 4933
5164 5165
5861 8278
5877 5948
6291 6295
6297 6307
6311 6327
6324 6325
6328 6335
6336 6338
6337 6344
6344 6350
6749 6753
6857 6867
7221 7250
7259 7787
7583 7584
7610 7620
7788 8101
8009 8178
8174 8199
8252 8595
8263 8276
8681 8770
8733 8744
8746 8773
9049 9160
9115 9150
9142 9150
9169 9211
9240 9446
9443 9594
9606 9658
9665 9759
9753 9821
9960 9981
9990 10002
10012 10146
10081 10083
10120 10132
10202 10562
10551 10562
10598 10694
10623 10633
10710 10744
10819 11164
11371 11441
11672 11672
11679 11680
11691 11691
11694 11704
11731 11796
11739 11980
12036 12067
12114 12142
12170 12321
12323 12351
12425 12468
12498 12514
12540 12739
12753 12805
12805 12921
12924 13097
12962 12962
13086 13117
13116 13247
13126 13126
13176 1318

### Get gene labels for first chromosome

In [32]:
chrom_size = 248956422

In [50]:
def get_gene_labels(ann_df, chrom_size, who_annotated):
    labels = pd.Series(np.zeros(chrom_size // 100))
              
    for ind, row in tqdm_notebook(ann_df.iterrows()):
        if row[2] == "CDS" and row[1] == who_annotated:
            start = ceil(row[3] / 100)
            stop = floor(row[4] / 100)
            labels[start:stop+1] = np.ones(stop - start + 1)
    return labels

In [51]:
gene_labels_ensembl = get_gene_labels(ann_df, chrom_size, "ensembl")

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

In [58]:
type(gene_labels_ensembl)

pandas.core.series.Series

In [61]:
gene_labels_ensembl.to_hdf("../Labels/ensembl_CDSs.hdf", "w")

In [54]:
sum(gene_labels_ensembl)

100472.0

In [55]:
len(gene_labels_ensembl)

2489564

In [31]:
print(gene_labels[:1000])

0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
5      0.0
6      1.0
7      1.0
8      1.0
9      1.0
10     1.0
11     1.0
12     1.0
13     1.0
14     1.0
15     1.0
16     1.0
17     1.0
18     1.0
19     1.0
20     1.0
21     1.0
22     1.0
23     1.0
24     1.0
25     1.0
26     1.0
27     1.0
28     1.0
29     1.0
      ... 
970    1.0
971    1.0
972    1.0
973    1.0
974    1.0
975    1.0
976    1.0
977    1.0
978    1.0
979    1.0
980    1.0
981    1.0
982    1.0
983    1.0
984    1.0
985    1.0
986    1.0
987    1.0
988    1.0
989    1.0
990    1.0
991    1.0
992    1.0
993    1.0
994    1.0
995    1.0
996    1.0
997    1.0
998    1.0
999    1.0
Length: 1000, dtype: float64
